# LQ fit for $\phi_1$ model

In [1]:
import os

In [2]:
os.getcwd()

'/home/aebischer/Repositories/LQfit'

In [30]:
import smelli
from treemodels import ModelPoint
import parameterscan
import numpy as np
import flavio
from wilson import Wilson
from scan.randompar import random_omega1
from smelli import GlobalLikelihood
import warnings

In [3]:
# from parameterscan import ScanStoreSQL
# store = ScanStoreSQL('my scan', datadir='.')

Tree level matching for the $\phi_1$ model

In [4]:
def LQ_match_tree(yomega1ql, yomega1eu, Momega1):
    #input is a numpy array for the couplings and a number for the masses
    p_dict = {'yomega1ql': [yomega1ql.tolist()], 'yomega1eu': [yomega1eu.tolist()], 
              'Momega1': [Momega1,Momega1,Momega1]}
    pt = ModelPoint(p_dict, Momega1)
    wc_tree = pt.wc
    return wc_tree.dict

In [7]:
#test
# yomega1ql = 50 * np.identity(3)
# yomega1eu = 90 * np.identity(3)
# Momega1 = 50
# p_dict = {'yomega1ql': [yomega1ql.tolist()], 'yomega1eu': [yomega1eu.tolist()], 
#           'Momega1': [Momega1,Momega1,Momega1]}
# pt = ModelPoint(p_dict, Momega1)
# LQ_match_tree(yomega1ql, yomega1eu, Momega1)

Setting up the scan

In [ ]:
import numpy as np

def random_LQ_couplings():
    return {'yomega1ql': np.random.rand(3,3), 'yomega1eu': np.random.rand(3,3), 
              'Momega1': np.random.rand()}

def LQ_obs(par):
    w = Wilson(LQ_match_tree(par['yomega1ql'], par['yomega1eu'], par['Momega1']), 160, 'SMEFT', 'Warsaw')
    obs_list = ['Rtaul(B->Dlnu)', 'BR(B+->pitaunu)', 'BR(B+->Knunu)', 'BR(B+->pinunu)', 'DeltaM_s', 'DeltaM_d', 
                'eps_K', 'x12_D', 'BR(K+->pinunu)', 'd_n', 'BR(tau->Knu)', 'BR(tau->pinu)']
    res = {obs : flavio.np_prediction(obs, w) for obs in obs_list}
    return res

In [ ]:
from parameterscan import RandomScan
scan = RandomScan(store,
                  parfunc=random_omega1,
                  funcdic={'observables': LQ_obs})
scan.run(batchsize=1, batches=10)

# $\lambda^R_{23}$ - $\lambda^L_{33}$ plot

In [16]:
def wc_func(x,y):
    yomega1ql = np.array([[0, 0, 0],
           [0, 0, 0],
           [0, 0, x]])
    yomega1eu = np.array([[0, 0, 0],
           [0, 0, y],
           [0, 0, 0]])
    Momega1 = 1000
    return LQ_match_tree(yomega1ql, yomega1eu, Momega1)

In [17]:
wc_func(1,2)

{'eu_2233': 6e-06,
 'lequ1_3233': 3e-06,
 'lequ3_3233': -7.5e-07,
 'lq1_3333': 7.5e-07,
 'lq3_3333': -7.5e-07}

In [11]:
def ll_func(x,y):
    yomega1ql = np.array([[0, 0, 0],
           [0, 0, 0],
           [0, 0, x]])
    yomega1eu = np.array([[0, 0, 0],
           [0, 0, y],
           [0, 0, 0]])
    Momega1 = 1000
    w = Wilson(LQ_match_tree(yomega1ql, yomega1eu, Momega1), 160, 'SMEFT', 'Warsaw')
    gl = smelli.GlobalLikelihood(eft='SMEFT', basis='Warsaw')
    glp = gl.parameter_point(w)
    return glp.log_likelihood_dict()

In [12]:
wc_func(1,2)

{'fast_likelihood_leptons.yaml': -36.86996783980792,
 'fast_likelihood_quarks.yaml': -0.872676508619179,
 'global': -inf,
 'likelihood_bcpv.yaml': 0.0014275384191959617,
 'likelihood_bqnunu.yaml': 0.04320029788437374,
 'likelihood_ewpt.yaml': -21.450327551677532,
 'likelihood_lept.yaml': 0.0013079768142074677,
 'likelihood_lfu_fccc.yaml': -0.021172495508544387,
 'likelihood_lfu_fcnc.yaml': 0.7070803921003215,
 'likelihood_lfv.yaml': -inf,
 'likelihood_rd_rds.yaml': 7.009586591037405,
 'likelihood_zlfv.yaml': -0.006826762115522911}

In [33]:
def run_plot(eft, scale, xmin, xmax, ymin, ymax,
        basis=None, steps=20, threads=1,
        setting='default', mpi=False,
        **kwargs):
    xmin = float(eval(str(xmin)))
    xmax = float(eval(str(xmax)))
    ymin = float(eval(str(ymin)))
    ymax = float(eval(str(ymax)))
    steps = int(steps)
    _x = np.linspace(xmin, xmax, steps)
    _y = np.linspace(ymin, ymax, steps)
    x, y = np.meshgrid(_x, _y)
    xy = np.array([x, y]).reshape(2, steps**2).T
    xy_enumerated = list(enumerate(xy))
#     from flavio_modifier import get_flavio
#     flavio = get_flavio(setting)
#     from smelli_modifier import get_global_likelihood
#     GlobalLikelihood = get_global_likelihood(setting)
#     if not 'exp_cov_folder' in kwargs:
#         kwargs['exp_cov_folder'] = os.path.join('./exp_covariances',setting)
    gl = GlobalLikelihood(eft=eft, basis=basis, **kwargs)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        keys_llh =  ['global'] + list(gl.log_likelihood_sm.keys())
        keys_obstable = list(gl.obstable_sm.keys())
    keys = keys_llh + keys_obstable
#     ###########################
#     #  compute chi2 on grid   #
#     ###########################
#     log_likelihood = partial(get_log_likelihood, gl=gl)
#     if mpi:
#         pool = MPIPool()
#         if not pool.is_master():
#             pool.wait()
#             sys.exit(0)
#     else:
#         pool = MultiPool(threads)
#     try:
#         ll_dict_list_enumerated = pool.map(log_likelihood, xy_enumerated)
#     except:
#         pool.close()
#         raise
#     pool.close()
#     if not mpi:
#         pool.join()
    ll_dict_list_enumerated = get_log_likelihood(xy_enumerated, gl=gl)
    ll_enumerated_dict = dict(list(ll_dict_list_enumerated))
    order_keys = sorted(ll_enumerated_dict.keys())
    ll_dict_list = [ll_enumerated_dict[k] for k in order_keys]
    plotdata = {}
    for k in keys:
        z = -2*np.array([ll_dict[k] for ll_dict in ll_dict_list]).reshape((steps, steps))
        plotdata[k] = {'x': x, 'y': y, 'z': z}
    ###########################
    return plotdata

In [34]:
def get_log_likelihood(xy_enumerated, gl):
    number, xy = xy_enumerated
    x, y = xy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        gp = gl.parameter_point(wc_func(x,y), _scale_func(x, y))
        ll_dict = gp.log_likelihood_dict()
    ll_dict.update({k: -copysign(v['pull SM']**2/2, v['pull SM'])
                    for k,v in gp._obstable_tree.items()})
    return (number,ll_dict)

def set_scale_function(scale):
    s = r"""
import numpy as np
def _scale_func(x, y):
    return np.float({})
""".format(scale)
    namespace = OrderedDict()
    exec(s, namespace)
    f = namespace.popitem()[1]
    global _scale_func
    _scale_func = f

In [35]:
run_plot('SMEFT', 'Warsaw', 0, 1, 0, 1, basis=None, steps=5, threads=1,setting='default', mpi=False)

ValueError: too many values to unpack (expected 2)